In this notebook we will look at the library we will use to annotate, as well as finding out how to analyze a corpus' vocabulary.

In [2]:
import scala.sys.process._ // since this is the first command for this JVM, it may take a few moments to complete

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

Let's look at the classes and interfaces we will be using.

In [4]:
val pipelineSrcDir = "nlp.spark.annotate/src/main/scala/nlp/spark/annotate/"

In [5]:
s"ls $pipelineSrcDir" !

Annotation.scala
Annotator.scala
Document.scala


The annotation class is at the heart here. These are the objects that we will craete to add ifnormatoin nito teh document with our NLP methods.

In [6]:
s"cat $pipelineSrcDir/Annotation.scala" !

package nlp.spark.annotate

/**
  * This class represents an annotation in text
  * @param aType the type of the annotation (e.g. token, sentence, etc.)
  * @param begin the index of the first character of the annotation
  * @param end the index after the last charactor of the annotation
  * @param metadata the metadata for the annotation (e.g. lemma, sentiment, etx.)
  */
case class Annotation(aType: String, begin: Int, end: Int, metadata: Map[String, String] = Map())

/**
  * Companion object for Annotation type
  */
object Annotation {
  /**
    * Annotation ordering
    * @tparam A Some annotation subtype (for our examples we will only use Annotation)
    * @return The [[Ordering]] of annotations
    */
  implicit  def orderingByName[A <: Annotation]: Ordering[A] = {
    Ordering.by(a => (a.begin, -a.end, a.aType))
  }
}

The document is the subject of our analys.

In [7]:
s"cat $pipelineSrcDir/Document.scala" !

package nlp.spark.annotate

import scala.collection.immutable.TreeSet

/**
  *
  * @param docName the name of the document
  * @param text the contents of the document
  * @param metadata the metadata associated with the document
  * @param annotations the annotations found in the document
  */
case class Document(
  docName: String,
  text: String,
  metadata: Map[String, String] = Map(),
  annotations: TreeSet[Annotation] = TreeSet())

The annotator is the trait which will be implementing to annotate our documents

In [8]:
s"cat $pipelineSrcDir/Annotator.scala" !

package nlp.spark.annotate

import scala.collection.immutable.TreeSet

/**
  * This trait represents an function that produces annotations from a given text or document
  */
trait Annotator {
  /**
    * This takes a text, optional metadata, and optional annotations and returns a sequence of new annotations
    * @param text the text to be annotated
    * @param metadata the metadata associated with the text
    * @param annotations the annotations alread found in the text
    * @return a sequence of new annotations found in the text
    */
  def annotate(
    text: String,
    metadata: Map[String, String] = Map(),
    annotations: TreeSet[Annotation] = TreeSet()): Seq[Annotation]

  /**
    * This takes a document and returns a sequence of new annotations
    * @param document the document to be annotated
    * @return a sequence of new annotations found in the text
    */
  def annotate(document: Document): Seq[Annotation] = annotate(document.text, document.metadata, document.annota

Now that we have looked at our code, let's load the jar files.

In [9]:
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/annotate-1.0.jar
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/opennlp-tools-1.6.0.jar

Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/annotate-1.0.jar
Finished download of annotate-1.0.jar
Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/opennlp-tools-1.6.0.jar
Finished download of opennlp-tools-1.6.0.jar


In [10]:
import nlp.spark.annotate.{ Annotation, Annotator, Document }

The dataset used in this Oriole is smaller for demo purposes. The full dataset can be found here: https://github.com/alexander-n-thomas/writingprompts

In [11]:
val wpData = sqlContext.read.json("writingprompts-master/writingprompts.sanitized.sample.json")

In [12]:
println(wpData.columns.mkString("\n"))

archived
author
author_flair_css_class
author_flair_text
body
controversiality
created_utc
distinguished
downs
edited
gilded
id
link_id
name
parent_id
retrieved_on
score
score_hidden
subreddit
subreddit_id
ups


In [13]:
println(wpData.first())

[true,prospectre,null,null,It was 7:30 in the evening. A little bit after you would start a task, and a little bit before you would go to sleep. It is the worst time for a guest. I do not like guests. I prefer to be alone. That is why I moved to these apartments. Size was not important. Sound proofing and privacy were. This way I could sit in my silent home and not be bothered. But the incessant pounding on the door to my apartment would not relent. I heard #BADWORDS#uely claims that they were police officers.

I open the door as far as the chain will allow. I see two men in standard issue police uniforms. The taller man holds up his badge and starts talking at me. I do not like to be talked at. He does not seem to care that it was now 7:31, and not a good time for me to have guests.

"Mr. Faulk?", the tall officer asked.

"Yes", I replied. I do not like talking with people. I want to go back inside.

"Mr. Faulk, there has been a lot of reports of missing people around these buildings.

Let's see who the most prolific authors are.

In [14]:
wpData.select("author").groupBy("author").count().orderBy(desc("count")).show(10)

+----------------+-----+
|          author|count|
+----------------+-----+
|    SurvivorType|   25|
|      DanKolar62|   25|
| Koyoteelaughter|   22|
| schoolgirlerror|   18|
|   LovableCoward|   16|
|    vonBoomslang|    8|
|    zoogreenjake|    8|
|  gallantgallade|    8|
|     AcheronFlow|    7|
|The_Eternal_Void|    7|
+----------------+-----+
only showing top 10 rows



Here we are creating a by-name-lookup for columns, so we can convert our rows into Documents.

In [15]:
val colNameToIdx = wpData.columns.zipWithIndex.toMap
val docNameIdx = colNameToIdx("id")
val textIdx = colNameToIdx("body")
val metadataColNameToIdx = Map() ++ colNameToIdx.filterKeys(colName => colName != "id" && colName != "body")

In [16]:
val docRdd = wpData.rdd.map {
  row =>
    val docName = row.getString(docNameIdx)
    val text = row.getString(textIdx)
    val metadata = metadataColNameToIdx.foldLeft(Map[String, String]()) { 
        case (md, (key, idx)) => 
        val value = row.get(idx)
        if (value != null) { // removing fields with null values
            md + (key -> value.toString)
        } else {
            md
        }
    }
    Document(docName, text, metadata)
}

In [17]:
docRdd.first()

Document(ceeh6gj,It was 7:30 in the evening. A little bit after you would start a task, and a little bit before you would go to sleep. It is the worst time for a guest. I do not like guests. I prefer to be alone. That is why I moved to these apartments. Size was not important. Sound proofing and privacy were. This way I could sit in my silent home and not be bothered. But the incessant pounding on the door to my apartment would not relent. I heard #BADWORDS#uely claims that they were police officers.

I open the door as far as the chain will allow. I see two men in standard issue police uniforms. The taller man holds up his badge and starts talking at me. I do not like to be talked at. He does not seem to care that it was now 7:31, and not a good tim...

In [18]:
val numDocs = docRdd.count()
numDocs

2744

Now let's define our annotators.

In [19]:
import nlp.spark.annotate.{ Annotation, Annotator, Document }
import opennlp.tools.stemmer.{ PorterStemmer => OpenNLPPorterStemmer }
import scala.collection.immutable.TreeSet
import scala.util.matching.Regex
import java.util

/**
  * This takes a regex and creates annotation for each match in the given document
  */
case class RegexTokenizer(regex: Regex) extends Annotator {
  private val annoType = "token"
  
  override def annotate(
    text: String,
    metadata: Map[String, String] = Map(),
    annotations: TreeSet[Annotation] = TreeSet()): Seq[Annotation] = {
      regex.findAllMatchIn(text).map {
        m =>
          Annotation(annoType, m.start, m.end)
      }.toSeq
    }
}

/**
  * This creates a stem annotation using OpenNLP's Porter stemmer
  */
case class PorterStemmer() extends Annotator {
  private val annoType = "stem"
  
  override def annotate(
    text: String,
    metadata: Map[String, String] = Map(),
    annotations: TreeSet[Annotation] = TreeSet()): Seq[Annotation] = {
      annotations.iterator.collect {
        case anno if anno.aType == "token" => 
          // Ideally, we would not be creating this object here. We would create a singleton in the 
          // companion object instead
          val stem = new OpenNLPPorterStemmer().stem(text.substring(anno.begin, anno.end).toLowerCase)
          Annotation(annoType, anno.begin, anno.end, Map("stem" -> stem))
      }.toSeq
    }
}

/**
  * This creates normalized tokens from non-punctuation tokens by removing the given patterns
  */
case class Normalizer(toRemove: Set[String]) extends Annotator {
  private val annoType = "ntoken"
  private val punctPattern = "\\p{Punct}+".r.pattern
  
  override def annotate(
    text: String,
    metadata: Map[String, String] = Map(),
    annotations: TreeSet[Annotation] = TreeSet()): Seq[Annotation] = {
      annotations.iterator.collect {
        case anno if anno.aType == "stem" && !punctPattern.matcher(anno.metadata("stem")).matches =>
          val stem = anno.metadata("stem")
          val normedToken = toRemove.foldLeft(anno.metadata("stem")) {
            case (nt, subs) => 
              nt.replaceAll(subs, "")
          }
          Annotation(annoType, anno.begin, anno.end, Map("ntoken" -> normedToken))
      }.toSeq
    }
}

/**
  * This is a simple implementation of a trie to be used for phrase matching.
  */
class Node[A](val children: util.HashMap[A, Node[A]] = new util.HashMap[A, Node[A]]()) extends Serializable {
  private var _maxDepth = 0
    
  def maxDepth = _maxDepth
    
  def contains(elem: Seq[A]): Boolean = {
    var cursor = this
    var stack = elem
    while (stack.nonEmpty && cursor.children.containsKey(stack.head)) {
      cursor = cursor.children.get(stack.head)
      stack = stack.tail
    }
    stack.isEmpty
  }
    
  def add(elem: Seq[A]): Unit = {
    var cursor: Node[A] = this
    var stack: Seq[A] = elem
    while (stack.nonEmpty) {
      if (!cursor.children.containsKey(stack.head)) {
        cursor.children.put(stack.head, new Node())
      }
      cursor = cursor.children.get(stack.head)
      stack = stack.tail
    }
    if (elem.size > _maxDepth) {
      _maxDepth = elem.size
    }
  }
    
  override def toString: String = s"Node($children)"
}

/**
  * This creates entity (phrase) annotations
  */
case class EntityAnnotator(trie: Node[String], tag: String) extends Annotator {
  private val annoType = "entity"
  
  override def annotate(
    text: String,
    metadata: Map[String, String] = Map(),
    annotations: TreeSet[Annotation] = TreeSet()): Seq[Annotation] = {
      annotations.iterator.filter {
        case anno => 
          anno.aType == "ntoken"
      }.sliding(trie.maxDepth).flatMap {
        window =>
          val matches = window.inits.filter {
            ntokens =>
              ntokens.nonEmpty && trie.contains(ntokens.map(_.metadata("ntoken")))
          }
          matches.map {
            m => 
              Annotation(annoType, m.head.begin, m.last.end, 
                Map(
                  "entity" -> m.map(_.metadata("ntoken")).mkString(" "),
                  "entity-tag" -> tag
                ))
          }
      }.toSeq
    }
}

Now that we have defined our annotators, let's create the basic ones.

In [20]:
val tokenizer = RegexTokenizer("[\\p{Alnum}']+|\\p{Punct}".r)
val stemmer = PorterStemmer()
val normalizer = Normalizer(Set("'s", "'$"))

Many stories in /r/WritingPrompts may be classified as science fiction or fantasy. So, I have gathered wordlists from the internet (primarily from https://myvocabulary.com)

In [21]:
"head -n 20 scifiwordlist.txt" !

aberration
abiosis
abolish
abrade
absolute magnitude
absolute zero
absorption
absorption lines
acceleration
accretion disk
accumulation
acid
acidic
activist
adaptation
adventure
affect
agonistic
agrarian
airborne


We need to make sure that our phrases are normalized in the same manner as our text.

In [22]:
val scifiTrie = new Node[String]()

val scifiEntities = scala.io.Source.fromFile("scifiwordlist.txt").getLines.map {
  phrase =>
    val phraseDoc = Document("", phrase)
    normalizer(stemmer(tokenizer(phraseDoc))).annotations.collect {
      case anno: Annotation if anno.aType == "ntoken" =>
        anno.metadata("ntoken")
    }.toSeq
}.toList

scifiEntities.foreach(scifiTrie.add(_))

val scifiAnno = EntityAnnotator(scifiTrie, "scifi")

In [23]:
val fantasyTrie = new Node[String]()

val fantasyEntities = scala.io.Source.fromFile("fantasywordlist.txt").getLines.map {
  phrase =>
    val phraseDoc = Document("", phrase)
    normalizer(stemmer(tokenizer(phraseDoc))).annotations.collect {
      case anno: Annotation if anno.aType == "ntoken" =>
        anno.metadata("ntoken")
    }.toSeq
}.toList

fantasyEntities.foreach(fantasyTrie.add(_))

val fantasyAnno = EntityAnnotator(fantasyTrie, "fantasy")

In [24]:
val annotated = docRdd.map {
  doc =>
    fantasyAnno(scifiAnno(normalizer(stemmer(tokenizer(doc)))))
}

In [25]:
annotated.flatMap(_.annotations).map(_.aType).map(x => (x, 1L)).reduceByKey(_ + _).collect()

Array((ntoken,509631), (entity,27943), (token,616122), (stem,616122))

In [26]:
annotated.first().annotations.filter(_.aType == "entity")

TreeSet(Annotation(entity,134,138,Map(entity -> time, entity-tag -> scifi)), Annotation(entity,261,266,Map(entity -> sound, entity-tag -> scifi)), Annotation(entity,509,512,Map(entity -> far, entity-tag -> scifi)), Annotation(entity,520,525,Map(entity -> chain, entity-tag -> scifi)), Annotation(entity,540,543,Map(entity -> see, entity-tag -> scifi)), Annotation(entity,740,744,Map(entity -> time, entity-tag -> scifi)), Annotation(entity,874,878,Map(entity -> back, entity-tag -> fantasy)), Annotation(entity,1383,1390,Map(entity -> strang, entity-tag -> scifi)), Annotation(entity,1391,1397,Map(entity -> nois, entity-tag -> scifi)), Annotation(entity,1568,1572,Map(entity -> back, entity-tag -> fantasy)), Annotation(e...

Now we need to create a DataFrame from our documents. Let's define the type.

In [27]:
val docType = 
  StructType(
    StructField("docName", StringType) ::
    StructField("text", StringType) ::
    StructField("metadata", MapType(StringType, StringType)) ::
    StructField("annotations", ArrayType(StructType(
      StructField("aType", StringType) ::
      StructField("begin", IntegerType) ::
      StructField("end", IntegerType) ::
      StructField("metadata", MapType(StringType, StringType)) :: Nil
    ))) :: Nil
  )

Now the DataFrame

In [28]:
val dataframe = sqlContext.createDataFrame(
  annotated.map {
    doc =>
      Row(
        doc.docName, 
        doc.text, 
        doc.metadata, 
        doc.annotations.map(a => Row(a.aType, a.begin, a.end, a.metadata)).toSeq
      )
  }, docType
).cache()

In [29]:
dataframe.count()

2744

Now let's register the table so we can use it through SparkSQL

In [30]:
dataframe.registerTempTable("annotated")

Now we will create a table containing the average TFIDF of the tokens.

TFIDF is a formula for weighting a word that rewards frequent occurrences within a document, and penalizes occurring in too many documents.

$$T = set\ of\ tokens,\ t \in T$$
$$D = set\ of\ documents,\ d \in D,\ N = |D|$$
$$n_t = \#\ documents\ that\ contain\ t$$
$$tf_{t,d} = \#\ times\ token\ t\ occurs\ in\ document\ d$$
$$avg(tf_{t}) = \frac{\sum_{d}tf_{t,d}}{n_{t}}$$
$$idf_{t}= \log(\frac{N}{n_t})$$
$$tf.idf_{t,d} = tf_{t,d} \times idf_{t}$$
$$avg(tf.idf_{t}) = \frac{\sum_{d}tf.idf_{t,d}}{n_{t}}$$

First we create a table of the tokens and the documents they occur in.

The function `explode` will turn expand the data to one annotation per row i.e. a document with k annotations will turn into k rows with `explode`

Once we've `explode`d the annotations we extract the normalized tokens.

In [31]:
val ntokens = sqlContext.sql("""
SELECT docName, annotation.metadata['ntoken'] as ntoken
FROM (
    SELECT docName, explode(annotations) as annotation
    FROM annotated
) t
WHERE annotation.aType = 'ntoken'
""")
ntokens.registerTempTable("ntokens")

Below the mapping between our SQL and our definition of TFIDF above is  

| variable            | SQL                                              |
|---------------------|--------------------------------------------------|
| $t$                 | `ntoken`                                         |
| $N$                 | `numDocs`                                        |
| $n_{t}$             | `count(distinct(docName))` ... `GROUP BY ntoken` |
| $\sum_{d}tf_{t,d}$  | `count(distinct(docName))` ... `GROUP BY ntoken` |

In [32]:
val ntokenTFIDF = sqlContext.sql(s"""
SELECT 
    ntoken, 
    log($numDocs / count(distinct(docName))) as idf, 
    count(docName) / count(distinct(docName)) as avg_tf
FROM ntokens
GROUP BY ntoken
ORDER BY avg_tf * idf DESC
""").cache()
ntokenTFIDF.registerTempTable("ntoken_tfidf")

Since the cell below is the first one to run after a number of lazy transformations, it may take a few moments to complete

In [33]:
%%SQL
SELECT count(*)
FROM ntoken_tfidf

+-----+
|  _c0|
+-----+
|15861|
+-----+



In [34]:
%%SQL
SELECT 
    min(idf) as min_idf, max(idf) as max_idf, mean(idf) as mean_idf, stddev(idf) as stddev_idf
FROM ntoken_tfidf

+-------------------+-----------------+-----------------+------------------+
|            min_idf|          max_idf|         mean_idf|        stddev_idf|
+-------------------+-----------------+-----------------+------------------+
|0.46418965938031576|7.917171988845776|6.805004404766642|1.4060873296108611|
+-------------------+-----------------+-----------------+------------------+



In [35]:
%%SQL
SELECT 
    min(avg_tf) as min_avg_tf, max(avg_tf) as max_avg_tf, 
    mean(avg_tf) as mean_avg_tf, stddev(avg_tf) as stddev_avg_tf
FROM ntoken_tfidf

+----------+----------+------------------+------------------+
|min_avg_tf|max_avg_tf|       mean_avg_tf|     stddev_avg_tf|
+----------+----------+------------------+------------------+
|       1.0|      52.0|1.3679637874067643|1.4600388365462755|
+----------+----------+------------------+------------------+



In [36]:
%%SQL
SELECT 
    min(idf * avg_tf) as min_avg_tfidf, max(idf * avg_tf) as max_avg_tfidf, 
    mean(idf * avg_tf) as mean_avg_tfidf, stddev(idf * avg_tf) as stddev_avg_tfidf
FROM ntoken_tfidf

+-----------------+------------------+-----------------+------------------+
|    min_avg_tfidf|     max_avg_tfidf|   mean_avg_tfidf|  stddev_avg_tfidf|
+-----------------+------------------+-----------------+------------------+
|2.323364748553355|411.69294341998034|9.316447284997338|11.386087845385566|
+-----------------+------------------+-----------------+------------------+



As can be expected, "stop words" are the words with the lowest $idf$ 

In [37]:
%%SQL
SELECT *
FROM ntoken_tfidf
ORDER BY idf

+------+-------------------+-----------------+
|ntoken|                idf|           avg_tf|
+------+-------------------+-----------------+
|   the|0.46418965938031576|15.61855072463768|
|    to| 0.5834955931880921|8.467668190725016|
|     a| 0.5867667770013736|7.587155963302752|
|     i| 0.5966450265730363|9.132362673726009|
|    it| 0.6517422655918228|5.775524475524476|
|   and| 0.7026675746946321|9.150110375275938|
|    of| 0.7189884117438327|7.435302916978309|
|  that| 0.7524516100739185|4.372776488785769|
|    in| 0.8122065405759334|5.376847290640394|
|    on| 0.8270951530696842|4.388333333333334|
+------+-------------------+-----------------+
only showing top 10 rows



We see a collection of odd words with highest $idf$, most likely these occur in only a single document.

In [38]:
%%SQL
SELECT *
FROM ntoken_tfidf
ORDER BY idf DESC

+---------+-----------------+------+
|   ntoken|              idf|avg_tf|
+---------+-----------------+------+
|quickster|7.917171988845776|   9.0|
|    sofia|7.917171988845776|   8.0|
|   vaccin|7.917171988845776|   9.0|
|   frigat|7.917171988845776|   9.0|
|     kalu|7.917171988845776|   8.0|
|   sirlum|7.917171988845776|   9.0|
|  lubbard|7.917171988845776|   9.0|
|    riser|7.917171988845776|   9.0|
|   angela|7.917171988845776|   9.0|
|     lyca|7.917171988845776|   9.0|
+---------+-----------------+------+
only showing top 10 rows



Words with a low $avg(tf)$ are also likely words that occur in only a single document.

In [39]:
%%SQL
SELECT *
FROM ntoken_tfidf
ORDER BY avg_tf

+---------+-----------------+------+
|   ntoken|              idf|avg_tf|
+---------+-----------------+------+
| disquiet|7.917171988845776|   1.0|
|thirstier|7.917171988845776|   1.0|
|commander|7.917171988845776|   1.0|
|      286|7.917171988845776|   1.0|
|   foxhol|7.917171988845776|   1.0|
|  denounc|7.917171988845776|   1.0|
| vestment|7.917171988845776|   1.0|
|       yw|7.917171988845776|   1.0|
| everpres|7.917171988845776|   1.0|
|      330|7.917171988845776|   1.0|
+---------+-----------------+------+
only showing top 10 rows



Words that have a high $avg(tf)$ are likely also unique across the corpus, but are also likely important to the story they are present in (e.g. proper nouns)

In [40]:
%%SQL
SELECT *
FROM ntoken_tfidf
ORDER BY avg_tf DESC

+---------+------------------+------+
|   ntoken|               idf|avg_tf|
+---------+------------------+------+
|    tyler| 7.917171988845776|  52.0|
|squidward| 7.917171988845776|  43.0|
|   zhanch| 7.917171988845776|  35.0|
|      998| 7.917171988845776|  34.0|
|     nbsp|6.8185597001776665|  33.0|
|     hank| 7.917171988845776|  30.0|
|    aerui| 7.917171988845776|  28.0|
|     krab| 7.917171988845776|  27.0|
|   howard| 7.917171988845776|  26.0|
|  winfrei| 7.917171988845776|  26.0|
+---------+------------------+------+
only showing top 10 rows



Words with a low $avg(tf.idf)$ are also likely stop words, or at least words that are sufficiently frequent as to be of little use.

In [41]:
%%SQL
SELECT *, avg_tf * idf as avg_tfidf
FROM ntoken_tfidf
ORDER BY avg_tf * idf

+------+------------------+------------------+------------------+
|ntoken|               idf|            avg_tf|         avg_tfidf|
+------+------------------+------------------+------------------+
| thank|2.0941260933627572|1.1094674556213018| 2.323364748553355|
|   thi|0.9032565140352482| 2.862410071942446|2.5854905433221176|
|   wai|1.7794449347595418| 1.511879049676026| 2.690305516915074|
|  good|1.9010148291474223|1.4195121951219511|2.6985137330824385|
|  well| 1.844127454745371|1.4930875576036866| 2.753443757315669|
|  like|1.2054315937895959|2.2907542579075426| 2.761347556089792|
|  make| 1.751754134614356|1.6134453781512605|2.8263596121508936|
|    if|1.4494732627414222| 1.950310559006211|2.8269230093217796|
|   too|1.8864867285845126|1.5072115384615385| 2.843334564477138|
|  much|2.0063753448052486|1.4200542005420054|2.8491617362546076|
+------+------------------+------------------+------------------+
only showing top 10 rows



Words with a high $avg(tf.idf)$ are likely to be proper nouns

In [42]:
%%SQL
SELECT *, avg_tf * idf as avg_tfidf
FROM ntoken_tfidf
ORDER BY avg_tf * idf DESC

+---------+------------------+------+------------------+
|   ntoken|               idf|avg_tf|         avg_tfidf|
+---------+------------------+------+------------------+
|    tyler| 7.917171988845776|  52.0|411.69294341998034|
|squidward| 7.917171988845776|  43.0| 340.4383955203684|
|   zhanch| 7.917171988845776|  35.0|277.10101960960213|
|      998| 7.917171988845776|  34.0|269.18384762075635|
|     hank| 7.917171988845776|  30.0|237.51515966537326|
|     nbsp|6.8185597001776665|  33.0|  225.012470105863|
|    aerui| 7.917171988845776|  28.0|221.68081568768173|
|     krab| 7.917171988845776|  27.0|213.76364369883595|
|   howard| 7.917171988845776|  26.0|205.84647170999017|
|  winfrei| 7.917171988845776|  26.0|205.84647170999017|
+---------+------------------+------+------------------+
only showing top 10 rows



If we look at words are more in the middle of the $idf$ range, we see more interesting words.

In [43]:
%%SQL
SELECT *, avg_tf * idf as avg_tfidf
FROM ntoken_tfidf
WHERE 
  idf > 4.0 AND idf < 5.0
ORDER BY avg_tf * idf DESC

+-------------+------------------+------------------+------------------+
|       ntoken|               idf|            avg_tf|         avg_tfidf|
+-------------+------------------+------------------+------------------+
|          amp|4.6982961639775755|               5.4| 25.37079928547891|
|         jack|  4.73911815849783|               3.5|16.586913554742406|
|         john| 4.420664427379296| 3.727272727272727|16.477021956595557|
|        angel| 4.584967478670572|3.3214285714285716| 15.22864198272726|
|         boss| 4.872649551122353|2.8095238095238093|13.689824929343754|
|writingprompt|  4.27958582911939| 3.026315789473684|12.951378167071837|
|       dragon|  4.82612953548746|2.6818181818181817|12.942801936080004|
|         cave| 4.921439715291784|              2.45|12.057527302464873|
|        sword|  4.22829253473184|              2.75| 11.62780447051256|
|            c|  4.82612953548746| 2.409090909090909| 11.62658479003797|
+-------------+------------------+-----------------

Let's save these results in case we want to use them later

In [44]:
if (new java.io.File("ntoken-tfidf.parquet").isDirectory) {
  "rm -rf ntoken-tfidf.parquet" !
}

In [45]:
ntokenTFIDF.write.parquet("ntoken-tfidf.parquet")

### Exercise 1. Write a stop word annotator and re-do the token analysis

The annotator should use a stop words list (one is provided) to create new annotations.

In [46]:
/**
  * This creates stop word annotations a given set of stop words
  */
case class StopwordAnnotator(stopwords: Set[String]) extends Annotator {
  private val annoType = "stopword"
  
  override def annotate(
    text: String,
    metadata: Map[String, String] = Map(),
    annotations: TreeSet[Annotation] = TreeSet()): Seq[Annotation] = ???
}

Now that you have an annotator, load the stop words

In [47]:
val stopwords = scala.io.Source.fromFile("stopwords.txt").getLines.map {
  line =>
    // your code goes here
}.toSet

### Solution

In [48]:
/**
  * This creates stop word annotations a given set of stop words
  */
case class StopwordAnnotator(stopwords: Set[String]) extends Annotator {
  private val annoType = "stopword"
  
  override def annotate(
    text: String,
    metadata: Map[String, String] = Map(),
    annotations: TreeSet[Annotation] = TreeSet()): Seq[Annotation] = {
      annotations.iterator.collect {
        case anno if anno.aType == "ntoken" && stopwords.contains(anno.metadata("ntoken")) =>
          Annotation(annoType, anno.begin, anno.end)
      }.toSeq
    }
}

In [49]:
val stopwords = scala.io.Source.fromFile("stopwords.txt").getLines.flatMap {
  line =>
    normalizer(stemmer(tokenizer(Document("", line.trim)))).annotations.collect {
      case anno if anno.aType == "ntoken" =>
        anno.metadata("ntoken")
    }
}.toSet

### End of exercise

In [50]:
stopwords.toSeq.sorted.foreach(println)

a
about
abov
across
after
again
against
all
almost
alon
along
alreadi
also
although
alwai
among
an
and
ani
anoth
anybodi
anyon
anyth
anywher
ar
area
aren't
around
as
ask
at
awai
b
back
badword
be
becam
becaus
becom
been
befor
began
behind
best
better
between
big
both
but
by
c
came
can
can't
cannot
case
certain
certainli
clear
clearli
come
could
d
did
differ
do
doe
don't
done
down
dure
e
each
earli
either
end
enough
even
evenli
ever
everi
everybodi
everyon
everyth
everywher
f
face
fact
far
felt
few
find
first
for
four
from
full
fulli
further
g
gave
gener
get
give
given
go
good
got
great
greater
greatest
group
h
ha
had
have
he
he'd
he'll
her
here
herself
hi
high
higher
highest
him
himself
how
howev
i
i'd
i'll
i'm
if
import
in
interest
into
is
isn't
it
itself
j
just
k
keep
kind
knew
know
known
l
larg
last
later
latest
least
less
let
like
long
longer
longest
m
made
mai
make
man
mani
me
member
men
might
more
most
mostli
mr
much
must
my
myself
n
necessari
need
never
new
newer
newest
next
no


In [51]:
val stopwordAnnotator = StopwordAnnotator(stopwords)

In [52]:
val annotated2 = annotated.map(stopwordAnnotator.apply)

In [53]:
val dataframe2 = sqlContext.createDataFrame(
  annotated2.map {
    doc =>
      Row(
        doc.docName, 
        doc.text, 
        doc.metadata, 
        doc.annotations.map(a => Row(a.aType, a.begin, a.end, a.metadata)).toSeq
      )
  }, docType
).cache()

Now that we have found the stop words we can re-do our vocabulary analysis.

In [54]:
val stopwordRemovedDF = sqlContext.createDataFrame(
  annotated2.map {
    doc =>
      val stopwordsRemoved = doc.annotations.filter {
        anno =>
          anno.aType == "ntoken" && 
            !doc.annotations.contains(Annotation("stopword", anno.begin, anno.end))
      }
      doc.copy(annotations = stopwordsRemoved)
  }.map {
    doc =>
      Row(
        doc.docName, 
        doc.text, 
        doc.metadata, 
        doc.annotations.map(a => Row(a.aType, a.begin, a.end, a.metadata)).toSeq
      )
  }, docType
).cache()

In [55]:
stopwordRemovedDF.registerTempTable("stopwordRemovedDF")

Now we will create a TFIDF table as we did before

In [56]:
val ntokens2 = sqlContext.sql("""
SELECT docName, annotation.metadata['ntoken'] as ntoken
FROM (
    SELECT docName, explode(annotations) as annotation
    FROM stopwordRemovedDF
) t
WHERE annotation.aType = 'ntoken'
""")
ntokens2.registerTempTable("ntokens2")

val ntokenTFIDF2 = sqlContext.sql(s"""
SELECT 
    ntoken, 
    log($numDocs / count(distinct(docName))) as idf, 
    count(docName) / count(distinct(docName)) as avg_tf
FROM ntokens2
GROUP BY ntoken
ORDER BY avg_tf * idf DESC
""").cache()
ntokenTFIDF2.registerTempTable("ntoken_tfidf2")

In [57]:
%%SQL
SELECT *
FROM ntoken_tfidf2
ORDER BY idf

+------+------------------+------------------+
|ntoken|               idf|            avg_tf|
+------+------------------+------------------+
|  time|1.3980247009053801|2.1828908554572273|
|  look| 1.493925025312256|2.5698051948051948|
|    ey|1.7751345832584196|2.1139784946236557|
|   dai| 1.869799809799498| 1.851063829787234|
|  hand|1.8961486394962495|2.1189320388349513|
| peopl|2.0478750757120014|1.9519774011299436|
|didn't|2.0563857653799102|1.8376068376068375|
|  head|2.0649695090713016|1.7844827586206897|
|  life|2.0707332137880514|1.8265895953757225|
| start|2.0941260933627572|1.5739644970414202|
+------+------------------+------------------+
only showing top 10 rows



Feel free to continue the analysis and/or improve on the stopword list.

Now we save our data to use it in the next notebook

In [58]:
if (new java.io.File("annotated-1.parquet").isDirectory) {
  "rm -rf annotated-1.parquet" !
}

In [59]:
// This is writing out our sample + all the annotations we created, so it may take a few moments to complete
dataframe2.write.parquet("annotated-1.parquet") 